# Add correlation matrices to Dataframe

In [1]:
import re
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

Load *df_parameter* DataFrame:

In [2]:
df_par = pd.read_pickle('df_parameter.p')
display(df_par.head())
df_par.shape

,gBB,gCC,gAB,gAC,gBC,path
0,-1.0,-1.0,-1.0,-1.0,-1.0,data/batch_001/run_00001
1,-1.0,-1.0,-1.0,-1.0,-0.8,data/batch_001/run_00002
2,-1.0,-1.0,-1.0,-1.0,-0.6,data/batch_001/run_00003
3,-1.0,-1.0,-1.0,-1.0,-0.4,data/batch_001/run_00004
4,-1.0,-1.0,-1.0,-1.0,-0.2,data/batch_001/run_00005


(18014, 6)

In [ ]:
df_par.loc[:, 'gBB'].unique()
df_par.loc[:, 'gBC'].unique()

In [ ]:
n_grid = 200  # size of grid

# create labels for columns, these correspond to the upper triangular of a matrix
cols_x1x2 = [f"x{j+1}_x{i+1}" for i in range(n_grid) for j in range(i+1)]

print(cols_x1x2[:10])

In [ ]:
def get_index(df_par, gBB, gCC, gAB, gAC, gBC):
    """Get index corresponding to a specific choice for the interaction parameters gXY
    
    Args:
        df_par (DataFrame): DataFrame of the parameter space
        gBB (float): interaction strength between B-B
        gCC (float): interaction strength between C-C
        gAB (float): interaction strength between A-B
        gAC (float): interaction strength between A-C
        gBC (float): interaction strength between B-C
    
    Returns:
        index (int) corresponding to input interaction parameters
    """
    
    mask = (
        (df_par.loc[:, 'gBB'] == gBB) &
        (df_par.loc[:, 'gCC'] == gCC) &
        (df_par.loc[:, 'gAB'] == gAB) &
        (df_par.loc[:, 'gAC'] == gAC) &
        (df_par.loc[:, 'gBC'] == gBC)
    )
    
    if all(~mask):
        print('No match found, return None')
    elif mask.sum() == 1:
        return np.where(mask)[0][0]
    else:
        print('More than one match found, return the first')
        return np.where(mask)[0][0]

## Task:

Load npz-files and add pixels as features

1. Create new dataframe with flattened npz-matrices as rows and x1-x2 values as columns
2. Concat parameter df and correlation-matrix df

## First attempt: Reading all at once (not working)

Load npz-files and add pixels as features

1. Create new dataframe with flattened npz-matrices as rows and x1-x2 values as columns
2. Concat parameter df and correlation-matrix df

## Second attempt: Store as pickles and concat

1. Load npz files and change to float32.
2. Correlation functions are symmetric, therefore, store only upper triangular.
3. Flatten correlation matrices, they denote the rows of the DataFrames.
4. Store DataFrames in chunks of 1000 as pickles.
5. Restart notebook to drop loaded memory.
6. Reload DataFrames and concat them to a big DataFrame.
7. Store it as a pickle.

1\. First check if storing the matrices with float16-type is still acceptable or we should go with float32.

In [ ]:
gBB, gCC, gAB, gAC, gBC = 1.0, 1.0, -1.0, 0.2, 0.2
mask = (
    (df_par.loc[:, 'gBB'] == gBB) &
    (df_par.loc[:, 'gCC'] == gCC) &
    (df_par.loc[:, 'gAB'] == gAB) &
    (df_par.loc[:, 'gAC'] == gAC) &
    (df_par.loc[:, 'gBC'] == gBC)
)
corrBC_example = np.load(df_par.loc[mask, 'path'].iloc[0] + '/correlation_fct_BC.npz')['corrBC']



xgrid = np.linspace(-2.5, 2.5, 200)
x, y = np.meshgrid(xgrid, xgrid)
corrBC_float64 = np.array(corrBC_example, dtype=np.float64)
corrBC_float32 = np.array(corrBC_example, dtype=np.float32)
corrBC_float16 = np.array(corrBC_example, dtype=np.float16)

print(np.abs(corrBC_float64 - corrBC_float32).max())
print(np.abs(corrBC_float64 - corrBC_float16).max())
print(np.abs(corrBC_float32 - corrBC_float16).max())


Float32 and float16 seems to be fine. However, continue with float32.

Steps: 2. - 4.
- flatten upper triangular and store DataFrames in chunks

In [ ]:
if False:  # Execute only once, to preserve memory
    corr_list = []
    count = 0
    for i, path_npz in enumerate(df_par.loc[:, 'path']):

        corrBC = np.load(path_npz + '/correlation_fct_BC.npz')['corrBC']
        
        # flatten the upper triangular and reduce precision
        corrBC_flat = corrBC[np.triu_indices(n_grid)].astype(np.float32)
 
        corr_list.append(corrBC_flat)

        if (i+1) % 1000 == 0:
            count += 1
            df_corr = pd.DataFrame(corr_list, columns=cols_x1x2)
            print(count, df_corr.shape)
            df_corr.to_pickle(f'data/df_corr_batch_{count:02d}.p')
            corr_list = []
    df_corr = pd.DataFrame(corr_list, columns=cols_x1x2)
    count += 1
    print(count, df_corr.shape)
    df_corr.to_pickle(f'data/df_corr_batch_{count:02d}.p')

Steps: 5. - 7.

- restart notebook, reload, concat and save as pickle

In [ ]:
df_corr_list = [pd.read_pickle(f'data/df_corr_batch_{count:02d}.p') for count in range(1, 20)]

df_corr = pd.concat(df_corr_list, axis=0)

In [ ]:
df_corr.shape

In [ ]:
df_corr.to_pickle(f'data/df_corr.p')

## Reduce correlation DataFrame further with PCA

1. Restart notebook and skip the above part to only load the df_corr.p into memory (otherwise it can happen that the kernel dies in the following, at least on my local laptop).
2. Standardize data and apply PCA.
3. Check that the applied PCA does not destroy relevant information.

Start with step 1:

In [3]:
df_corr = pd.read_pickle('data/df_corr.p')

Continue with step 2: Use Principle Component analysis to reduce the number of columns further.

In [4]:
n_comp = 10

pipe_pca = Pipeline([
    ('std', StandardScaler()),
    ('pca', PCA(n_components=n_comp, random_state=42))
])

arr_corr = pipe_pca.fit_transform(df_corr)

Check results (steps 3)

In [ ]:
expl_var = pipe_pca.named_steps['pca'].explained_variance_ratio_

print('Explained variance:', sum(expl_var))

Explained variance for n_comp=2:  0.8722

Explained variance for n_comp=5:  0.9885

Explained variance for n_comp=10: 0.9985

Explained variance for n_comp=15: 0.9996

Transform back and invert Standardscaler:

In [10]:
print('arr_corr:', arr_corr.shape, 'principal components')
print(pipe_pca.named_steps['pca'].components_.shape, 'pca coefficients')


corr_back_scaled = np.dot(arr_corr, pipe_pca.named_steps['pca'].components_)

corr_backscaled = pipe_pca.named_steps['std'].inverse_transform(corr_back_scaled)

# needed to reconstruct decomposed correlation matrices
np.savez('corr_mat_backscaled.npz', corr_backscaled)

arr_corr: (18014, 10) principal components
(10, 20100) pca coefficients


Reconstruct correlation matrix:

In [ ]:
def plot_comparison_corr_mat(df_par, corr_backscaled, index, bool_save=False):
    """Reconstruct correlation matrix from the PCA and compare to the original one.
        
    Args:
        df_par (DataFrame): DataFrame containing the paths to of original correlation matrices
        corr_backscaled (numpy.array): Get matrix where rows are flattened correlation
                                       matrices with only upper trinagualr entries being stored.
        index (int): index of the individual correlation matrix
        bool_save (boolean): determines whether to save the figure, default=False
        
    Return:
        None
    
    """

    corr_mat_ori = np.load(df_par.loc[index, "path"] + '/correlation_fct_BC.npz')['corrBC']

    n_grid = corr_mat_ori.shape[0]
    
    
    corr_flat_pca = corr_backscaled[index, :]

    corr_mat_pca_half = np.zeros((n_grid, n_grid))
    corr_mat_pca_half[np.triu_indices(n_grid)] = corr_flat_pca

    corr_mat_pca = np.rot90(np.fliplr(corr_mat_pca_half)) + corr_mat_pca_half - np.diag(np.diag(corr_mat_pca_half))
    
    


    #===========================================================================
    # Define grid structure
    plt.rc('text', usetex=True)
    fig = plt.figure(figsize=(8, 3), dpi=300)

    fig.subplots_adjust(left=0.15, right=0.89, top=0.92, bottom=0.12)


    canv = gridspec.GridSpec(1, 2, width_ratios=[2.1, 1], wspace=0.4)
    canv_left = gridspec.GridSpecFromSubplotSpec(
        1, 2, canv[0, 0], width_ratios=[1, 1], wspace=0.1)

    ax1 = plt.subplot(canv_left[0, 0], aspect=1)
    ax2 = plt.subplot(canv_left[0, 1], aspect=1)    
    ax3 = plt.subplot(canv[0, 1], aspect=1)

    
    #===========================================================================
    # make plots
    x_grid = np.linspace(-100, 100, n_grid)
    
    x, y = np.meshgrid(x_grid, x_grid)
    ax1.set_title('$\mathcal{C}_{\mathrm{ori}}$')
    ax2.set_title(r'$\mathcal{C}_{\mathrm{pca}}$'+ f', n={n_comp}')
    ax3.set_title(r'$|\mathcal{C}_{\mathrm{pca}} - \mathcal{C}_{\mathrm{ori}}|$')
    
    vmin, vmax = corr_mat_ori.min(), corr_mat_ori.max()
    im = ax1.pcolormesh(x, y, corr_mat_ori, shading='auto')
    im.set_rasterized(True)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="9%", pad=0.05)
    cax.set_axis_off()
    
    im = ax2.pcolormesh(x, y, corr_mat_pca, shading='auto', vmin=vmin, vmax=vmax)
    im.set_rasterized(True)
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="9%", pad=0.05)
    cbar = fig.colorbar(im, cax=cax, extend='both')
    
    im = ax3.pcolormesh(x, y, np.abs(corr_mat_ori - corr_mat_pca), shading='auto', cmap='bwr')
    im.set_rasterized(True)
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes("right", size="9%", pad=0.05)
    cbar = fig.colorbar(im, cax=cax)
    
    ax1.set_xlabel(r'$x_1$')
    ax2.set_xlabel(r'$x_1$')
    ax1.set_ylabel(r'$x_2$')
    ax3.set_xlabel(r'$x_1$')
    ax2.set_yticklabels([])
    
    # make title
    title_str = ''
    title_str += r'$g_{BB}=' + str(df_par.loc[index, 'gBB']) + '$, '
    title_str += r'$g_{CC}=' + str(df_par.loc[index, 'gCC']) + '$, '
    title_str += r'$g_{AB}=' + str(df_par.loc[index, 'gAB']) + '$, '
    title_str += r'$g_{AC}=' + str(df_par.loc[index, 'gAC']) + '$, '
    title_str += r'$g_{BC}=' + str(df_par.loc[index, 'gBC']) + '$'
    
    ax2.annotate(
        title_str,
        xy=(0.6, 1.25),
        xycoords='axes fraction',
        ha='center',
    )
    
    for ax in [ax1, ax2, ax3]:
        ax.set_xticks([-100, 0, 100])
        ax.set_yticks([-100, 0, 100])
        

    if bool_save:
        fig_name = f'figures/corr_ind{index}_pca_n_{n_comp}.pdf'
        
        plt.savefig(fig_name)
        plt.close()

        if fig_name[-3:] == 'png':
            subprocess.check_output(["convert", fig_name, "-trim", fig_name])
        elif fig_name[-3:] == 'pdf':
            subprocess.check_output(["pdfcrop", fig_name, fig_name])


    return None

In [ ]:
index = get_index(df_par, 0.2, 0.2, 0.2, 0.2, -0.2)

plot_comparison_corr_mat(df_par, corr_backscaled, index=index, bool_save=True)

In the above analysis we find that 10 principal components can explain the variance to 99.85%.
We proceed with this value.

### Merge df_corr and df_par

In [5]:
df_corr_pca = pd.DataFrame(arr_corr, columns=[f'pca comp {i}' for i in range(1, n_comp+1)])

In [6]:
df_main = pd.merge(
    left=df_par,
    right=df_corr_pca,
    how='inner',
    left_index=True,
    right_index=True
)
df_main.head()

,gBB,gCC,gAB,gAC,gBC,path,pca comp 1,pca comp 2,pca comp 3,pca comp 4,pca comp 5,pca comp 6,pca comp 7,pca comp 8,pca comp 9,pca comp 10
0,-1.0,-1.0,-1.0,-1.0,-1.0,data/batch_001/run_00001,-105.659465,37.965591,75.525219,0.440110,71.498446,-6.941616,-0.005033,24.897444,-19.683921,1.891975
1,-1.0,-1.0,-1.0,-1.0,-0.8,data/batch_001/run_00002,-102.790897,32.000799,59.039534,0.308656,49.503230,-9.023706,0.041075,15.930131,-10.015480,0.990542
2,-1.0,-1.0,-1.0,-1.0,-0.6,data/batch_001/run_00003,-98.020880,24.320418,41.510031,0.182524,29.262609,-9.871892,0.070699,8.940829,-2.802803,0.314344
3,-1.0,-1.0,-1.0,-1.0,-0.4,data/batch_001/run_00004,-90.632865,14.695513,23.230042,0.067592,11.804668,-9.495908,0.081263,4.117469,1.445592,-0.088344
4,-1.0,-1.0,-1.0,-1.0,-0.2,data/batch_001/run_00005,-79.346890,4.165145,4.233191,-0.034407,-0.536990,-8.301869,0.071124,1.586937,2.300852,-0.179525


In [7]:
df_main.shape

(18014, 16)

In [8]:
df_main.to_pickle(f'df_main.p')